In [ ]:
import subprocess

In [2]:
subprocess.run(['pwd'])

CompletedProcess(args=['pwd'], returncode=0)

In [1]:
subprocess.run('run_array_job_test_model.sh')

FileNotFoundError: [Errno 2] No such file or directory: 'run_array_job_test_model.sh': 'run_array_job_test_model.sh'

In [1]:
import PIL.Image as Image
import PIL.ImageDraw as ImageDraw
import math
import random
import numpy as np
import os
import pandas as pd

(1, 300, 5)

(300, 5)

In [16]:

my_rois = np.load('my_rois.npy') 
my_rois = my_rois[0,:,:]
# image = Image.new("RGB", (600, 800))
image = Image.open('images/cat09image0001.png')
draw = ImageDraw.Draw(image)
num_box = my_rois.shape[0] #300
min_rgb = 20
max_rgb = 256
# rs = np.arange(50, 256, (256-50)//(num_box/3)).astype(int)
# gs = np.arange(50, 256, (256-50)//(num_box/3)).astype(int)
# bs = np.arange(50, 256, (256-50)//(num_box/3)).astype(int)

# gs = np.repeat(0, num_box)
# bs = np.repeat(0, num_box)
small_rois = np.ndarray((1,4), dtype="float32")
for iB in range(num_box-1):
    if my_rois[iB, 3] - my_rois[iB, 1] < 80 and my_rois[iB, 3] - my_rois[iB, 1] < 80:
        small_rois = np.vstack((small_rois, my_rois[iB, 1:5]))
small_rois = small_rois[1:,:]        
num_small_rois = small_rois.shape[0] #56
rs = np.array(random.sample(range(min_rgb, max_rgb), num_small_rois))
gs = np.array(random.sample(range(min_rgb, max_rgb), num_small_rois))
bs = np.array(random.sample(range(min_rgb, max_rgb), num_small_rois))

for iB in range(num_small_rois-1):
    draw.rectangle(small_rois[iB, 0:4], outline = (rs[iB], gs[iB], bs[iB]))
image.show()
image.save('temp.png')

In [8]:
my_rois.shape

(300, 5)

In [10]:
small_rois.shape

(56, 4)

In [11]:
small_rois[0,0].type

AttributeError: 'numpy.float64' object has no attribute 'type'

In [6]:
my_rois[0,:]

array([  0.     , 153.92946, 192.11487, 599.     , 599.     ],
      dtype=float32)